`pip install llama-index-llms-openai`  
`pip install llama-index-graph-stores-neo4j`   
`pip install llama-index-embeddings-openai`  
`pip install neo4j`  

Download neo4j desktop client using https://neo4j.com/download/ or use neo4j aura with https://neo4j.com/cloud/platform/aura-graph-database/  
Create a new project, set the password and select Plugins to install APOC to the database. 

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

import logging
import sys
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# define LLM
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.llm = llm
Settings.chunk_size = 512

In [3]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore


from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [4]:
documents = SimpleDirectoryReader(
    "data/harry_potter"
).load_data()

In [5]:
username = "neo4j"
password = ""
url = ""
database = "neo4j"

In [7]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

In [16]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
)

In [19]:
response = await query_engine.aquery(
    "Tell me about Harry Potter",
)
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Harry Potter is a character who had various interactions with Hagrid, such as being brought by him, watched by him, and helped by him. Harry Potter also wore round glasses, slept in a cupboard under the stairs, and moved into Dudley's second bedroom. Additionally, Harry Potter learned from Uncle Vernon, realized something about his mouth, and felt both sure and strangely about certain situations.</b>